# RAGProRetriever

## Overview
This retriever allows you to filter your search by domain and provides higher quality results than just chunks of context; meaning that the need to perform Corrective RAG or worry about low-quality data is gone.

Instead, this retriever uses a search engine to find relevant documents with an LLM to provide your RAG app with the highest quality data possible.

We are also the first retriever to support url (domain) filtering which means you no longer need to scrape and index specific sites yourself.

In [62]:
%pip install --upgrade langchain-community langchain


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Get your api key [here](https://docs.rag.pro/account)

In [63]:
from langchain_community.retrievers import RAGProRetriever

retriever = RAGProRetriever(
    api_key="RAG-PRO-API-KEY",
    # Choose between small, medium, or large model sizes depending on your use case
    model="small",
    include_sources=True
)

## Usage

In [64]:
docs = retriever.invoke("Your query here")

print(docs[0].page_content[:400])

KeyboardInterrupt: 

In [ ]:
# Define your query and source filter
query = "How can I create a react agent graph with openai as the llm?"
source_filter = ["https://python.langchain.com/api_reference/", "https://langchain-ai.github.io/langgraph/#example"]

# Retrieve documents
docs = retriever.get_relevant_documents(query, urls=source_filter)

# Print the full content of each document
for i, doc in enumerate(docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content)  # This will print the entire content
    print(f"Source: {doc.metadata.get('source', 'Not available')}")
    print("-" * 50)  # Separator between documents



Document 1:
To create a ReAct agent graph using OpenAI as the Large Language Model (LLM) in LangGraph, you can follow these steps:

## Install Required Packages
Ensure you have the necessary packages installed. You can do this using the following command:

```python
%pip install -U langgraph langchain-openai
```

## Set Up API Keys
Set your OpenAI API key as an environment variable:

```python
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
```

## Initialize the Model
Initialize the OpenAI model you want to use:

```python
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4", temperature=0)
```

## Define Tools (Optional)
If you want to use tools with your agent, define them using the `@tool` decorator:

```python
from typing import Literal
from langchain_core.tools import tool

@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to g

## Use within a chain
Like other retrievers, `RAGProRetriever` can be integrated into LLM applications via chains.

We will need a LLM or chat model:

In [ ]:
%pip install langchain-openai

  Using cached langchain_openai-0.2.2-py3-none-any.whl (49 kB)
  Using cached openai-1.51.2-py3-none-any.whl (383 kB)
  Using cached tiktoken-0.8.0-cp310-cp310-macosx_11_0_arm64.whl (982 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl (78 kB)
  Using cached distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached jiter-0.6.1-cp310-cp310-macosx_11_0_arm64.whl (301 kB)
  Using cached regex-2024.9.11-cp310-cp310-macosx_11_0_arm64.whl (284 kB)

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

## Simple Chain with Sources

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_template(
    """
    Answer the question based only on the context provided.
    Context: {context}
    Question: {question}
    """
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def format_sources(docs):
    sources = []
    for doc in docs:
        source = doc.metadata.get('source', 'Unknown')
        if isinstance(source, list):
            sources.extend(source)
        else:
            sources.append(source)
    return "\n".join(set(sources))

def retrieve_with_sources(inputs):
    query = inputs["question"]
    urls = inputs.get("urls", None)
    return retriever.get_relevant_documents(query, urls=urls)

def process_llm_response(inputs):
    llm_response = inputs["llm_response"]
    sources = inputs["sources"]
    return f"{llm_response}\n\nSources:\n{sources}"

chain = (
    RunnablePassthrough()
    | {
        "context": RunnableLambda(retrieve_with_sources) | RunnableLambda(format_docs),
        "question": lambda x: x["question"],
        "sources": RunnableLambda(retrieve_with_sources) | RunnableLambda(format_sources)
    }
    | {
        "llm_response": (lambda x: {"context": x["context"], "question": x["question"]}) | prompt | llm | StrOutputParser(),
        "sources": lambda x: x["sources"]
    }
    | RunnableLambda(process_llm_response)
)

# Example usage of the chain with filtered sources
result = chain.invoke({
    "question": "How can I create a react agent graph with openai as the llm?",
    "urls": ["https://python.langchain.com/api_reference/", "https://langchain-ai.github.io/langgraph/#example"]
})
print(result)

To create a ReAct agent graph using OpenAI as the Large Language Model (LLM), follow these steps:

1. **Setup and Dependencies**: Install the necessary packages and set up your OpenAI API key.
   ```bash
   %pip install -U langgraph langchain-openai
   ```

   ```python
   import getpass
   import os

   def _set_env(var: str):
       if not os.environ.get(var):
           os.environ[var] = getpass.getpass(f"{var}: ")

   _set_env("OPENAI_API_KEY")
   ```

2. **Define the Model and Tools**: Load the OpenAI chat model and define any tools you want to use.
   ```python
   from langchain_openai import ChatOpenAI
   from langchain_core.tools import tool

   model = ChatOpenAI(model="gpt-3.5-turbo")

   @tool
   def get_weather(city: str):
       """Use this to get weather information."""
       if city == "nyc":
           return "It might be cloudy in nyc"
       elif city == "sf":
           return "It's always sunny in sf"
       else:
           raise AssertionError("Unknown city")

  

## Agentic RAG with RAG Pro Retriever

Taken from this [Notebook](https://github.com/langchain-ai/langchain/blob/master/cookbook/langgraph_agentic_rag.ipynb)

In [ ]:
%pip install langgraph langchainhub


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [50]:
from typing import (
    Annotated,
    Sequence,
    TypedDict,
)
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    """The state of the agent."""

    # add_messages is a reducer
    # See https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [66]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from typing import List, Optional


model = ChatOpenAI(model="gpt-4o-mini")

@tool
def search_langchain(query: str, urls: Optional[List[str]] = None):
    """
    Agent with access to up-to-date information from Langchain and Langgraph's python examples, documentation, and guides.

    Args:
        query (str): The search query.
        urls (Optional[List[str]]): List of URLs to filter the search results.
                                    If not provided, the search will not be filtered by URL.
    """
    return retriever.get_relevant_documents(query, urls=['https://python.langchain.com/docs/', 'https://langchain-ai.github.io/'])


tools = [search_langchain]

model = model.bind_tools(tools)

In [67]:
from langgraph.prebuilt import create_react_agent

graph = create_react_agent(model, tools=tools)

In [68]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [69]:
inputs = {"messages": [("user", "Create a Math expert react agent graph with openai as the llm that breaks down user's math queries into smaller steps and uses python/calculator tools to accomplish them")]}
print_stream(graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Create a Math expert react agent graph with openai as the llm that breaks down user's math queries into smaller steps and uses python/calculator tools to accomplish them
================================== Ai Message ==================================
Tool Calls:
  search_langchain (call_uUDgcUy6KPSvCpPj4Lw8vlfw)
 Call ID: call_uUDgcUy6KPSvCpPj4Lw8vlfw
  Args:
    query: Math expert react agent graph with OpenAI LLM
================================= Tool Message =================================
Name: search_langchain

[Document(metadata={'source': ['https://python.langchain.com/docs/additional_resources/arxiv_references/', 'https://python.langchain.com/docs/integrations/llms/google_vertex_ai_palm/', 'https://python.langchain.com/docs/integrations/document_transformers/markdownify/']}, page_content='To create a math expert React agent graph using OpenAI\'s LLM, you can leverage the concepts and tools discu

## API reference

For detailed documentation of all `RAGProRetriever` features and configurations, refer to the [API reference](https://docs.rag.pro/documentation).
